## Tratamento do PPA por regionalização

Nesse notebook são tratados os dados do Plano Plurianual por regionalização (2022 - 2025) da Prefeitura Municipal de São Paulo

In [1]:
import pandas as pd
from utils.load_csv import load_csv
from utils.save_csv import save_csv

In [2]:
df_ppa_reg = load_csv("ppa_reg.csv")

#### Criação "id"

Cria variável "id" concatenando variáveis de código da base de dados para "chave" do merge

In [3]:
df_ppa_reg["id"] = df_ppa_reg[["codigo orgao",
                               "codigo unidade",
                               "codigo programa",
                               "codigo acao"]].apply(lambda x: "_".join(x.astype(str)), axis = 1)

In [4]:
cols = ["id"] + [col for col in df_ppa_reg.columns if col != "id"]

df_ppa_reg = df_ppa_reg[cols]

#### Transformação das variáveis de planejamento orçamentário

Transforma as variáveis de planejamento orçamentário da base de dados de string para float

In [5]:
cols_plan_recurso = ["valor 2022",
                     "valor 2023",
                     "valor 2024",
                     "valor 2025"]

In [6]:
for col in cols_plan_recurso:
    df_ppa_reg[col] = (df_ppa_reg[col]
                         .str.replace(".", "")
                         .str.replace(",", ".")
                         .astype(float))

#### Filtro regionalizável

Filtra a base de dados para conter apenas despesas regionalizáveis

In [7]:
supra_subprefs = ["Supra Subprefeitura",
                  "Supra Subprefeitura Leste",
                  "Supra Subprefeitura Oeste",
                  "Supra Subprefeitura Norte",
                  "Supra Subprefeitura Sul",
                  "Supra Subprefeitura Centro"]

In [8]:
df_ppa_reg = df_ppa_reg[~df_ppa_reg["descricao prefeitura regional"].isin(supra_subprefs)]

#### Normalização das subprefeituras

Trata os nomes das subprefreituras na base de dados para estar igual aos nomes do shapefile das subprefeituras

In [9]:
mapper = {"Subprefeitura Sé": "SE",
          "Subprefeitura Vila Mariana": "VILA MARIANA",
          "Subprefeitura Santana/Tucuruvi": "SANTANA-TUCURUVI",
          "Subprefeitura Campo Limpo": "CAMPO LIMPO",
          "Subprefeitura Itaim Paulista": "ITAIM PAULISTA",
          "Subprefeitura Pirituba/Jaraguá": "PIRITUBA-JARAGUA",
          "Subprefeitura Sapopemba": "SAPOPEMBA",
          "Subprefeitura Capela do Socorro": "CAPELA DO SOCORRO",
          "Subprefeitura Lapa": "LAPA",
          "Subprefeitura Jaçanã/Tremembé": "JACANA-TREMEMBE",
          "Subprefeitura Penha": "PENHA",
          "Subprefeitura São Miguel Paulista": "SAO MIGUEL",
          "Subprefeitura Perus/Anhanguera": "PERUS",
          "Subprefeitura Butantã": "BUTANTA",
          "Subprefeitura de Guaianases": "GUAIANASES",
          "Subprefeitura São Mateus": "SAO MATEUS",
          "Subprefeitura Cidade Ademar": "CIDADE ADEMAR",
          "Subprefeitura de Vila Prudente": "VILA PRUDENTE",
          "Subprefeitura Ipiranga": "IPIRANGA",
          "Subprefeitura Pinheiros": "PINHEIROS",
          "Subprefeitura Casa Verde/Cachoeirinha": "CASA VERDE-CACHOEIRINHA",
          "Subprefeitura Freguesia/Brasilândia": "FREGUESIA-BRASILANDIA",
          "Subprefeitura M'Boi Mirim": "M BOI MIRIM",
          "Subprefeitura Vila Maria/Vila Guilherme": "VILA MARIA-VILA GUILHERME",
          "Subprefeitura Aricanduva/Formosa/Carrão": "ARICANDUVA-FORMOSA-CARRAO",
          "Subprefeitura Santo Amaro": "SANTO AMARO",
          "Subprefeitura Mooca": "MOOCA",
          "Subprefeitura Ermelino Matarazzo": "ERMELINO MATARAZZO",
          "Subprefeitura Cidade Tiradentes": "CIDADE TIRADENTES",
          "Subprefeitura Itaquera": "ITAQUERA",
          "Subprefeitura Jabaquara": "JABAQUARA",
          "Subprefeitura Parelheiros": "PARELHEIROS"}

In [10]:
df_ppa_reg.loc[:, "descricao prefeitura regional"] = df_ppa_reg["descricao prefeitura regional"].map(mapper)

#### Salvar

In [11]:
pd.set_option("display.max_columns", None)

df_ppa_reg.sample(10)

,id,codigo orgao,descricao orgao,codigo unidade,descricao unidade,codigo programa,descricao programa,codigo acao,descricao acao,codigo da,descricao da,codigo meta,descricao meta,codigo regiao,descricao regiao,codigo prefeitura regional,descricao prefeitura regional,codigo distrito,descricao distrito,codigo sequencial da,codigo produto,descricao produto,descricao unidade medida,quantidade 2022,valor 2022,quantidade 2023,valor 2023,quantidade 2024,valor 2024,quantidade 2025,valor 2025
425,12_10_3016_4301,12,Secretaria Municipal das Subprefeituras,10,Gabinete do Secretário,3016,Nutrição e Promoção da Segurança Alimentar,4301,Ações Municipais de Abastecimento,777.04.19.00.001,Subprefeitura Capela do Socorro,777,Despesa Regionalizável,4,Sul,19,CAPELA DO SOCORRO,0,Supra-Distrital,1,2874,Ações sanitárias,ações,"0,00",174103.0,"0,00",130357.0,0,140979.0,0,153160.0
1295,16_10_3010_2857,16,Secretaria Municipal de Educação,10,Gabinete do Secretário,3010,"Manutenção, Desenvolvimento e Promoção da Qual...",2857,Remuneração dos Profissionais do Magistério - ...,777.03.08.00.001,Subprefeitura Lapa,777,Despesa Regionalizável,3,Oeste,8,LAPA,0,Supra-Distrital,1,2616,Profissionais do Magistério (EF) remunerados,pesso,"398,00",30672923.0,"402,00",32026232.0,407,33105950.0,407,34329587.0
2623,19_10_3017_4570,19,Secretaria Municipal de Esportes e Lazer,10,Gabinete do Secretário,3017,"Promoção de Atividades Esportivas, Recreativas...",4570,Manutenção e Operação dos Pólos Regionais de E...,777.02.24.00.001,Subprefeitura Itaim Paulista,777,Despesa Regionalizável,2,Leste,24,ITAIM PAULISTA,0,Supra-Distrital,1,2650,Polos olímpicos e/ou de esporte de base implan...,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
6327,29_10_3024_2574,29,Secretaria Municipal de Urbanismo e Licenciamento,10,Secretaria Municipal de Urbanismo e Licenciame...,3024,Suporte Administrativo,2574,Gestão do Patrimônio Imobiliário Municipal,777.02.23.00.001,Subprefeitura São Miguel Paulista,777,Despesa Regionalizável,2,Leste,23,SAO MIGUEL,0,Supra-Distrital,1,2863,Manutenção das unidades em operação,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
1973,16_10_3025_3359,16,Secretaria Municipal de Educação,10,Gabinete do Secretário,3025,Proteção e Desenvolvimento Integral na Primeir...,3359,Construção de Centros de Educação Infantil - CEI,777.03.11.00.001,Subprefeitura Pinheiros,777,Despesa Regionalizável,3,Oeste,11,PINHEIROS,0,Supra-Distrital,1,2626,CEIs construídos,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
11079,98_25_3001_5425,98,Fundo de Desenvolvimento Urbano,25,FUNDURB - Secretaria Municipal de Cultura,3001,Acesso à Cultura,5425,Implantação da Casa de Cultura Cidade Ademar,777.04.20.00.001,Subprefeitura Parelheiros,777,Despesa Regionalizável,4,Sul,20,PARELHEIROS,0,Supra-Distrital,1,2978,Projeto concluído,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
3107,22_10_3008_5013,22,Secretaria Municipal de Infraestrutura Urbana ...,10,Gabinete do Secretário,3008,Gestão dos Riscos e Promoção da Resiliência a ...,5013,Intervenções no Sistema de Drenagem,777.02.24.00.001,Subprefeitura Itaim Paulista,777,Despesa Regionalizável,2,Leste,24,ITAIM PAULISTA,0,Supra-Distrital,1,2978,Projeto concluído,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
2951,20_50_3009_1096,20,Secretaria Municipal de Mobilidade e Trânsito,50,Secretaria Executiva de Transporte e Mobilidad...,3009,Melhoria da Mobilidade Urbana Universal,1096,"Acessibilidade, Ampliação, Reforma e Requalifi...",777.02.30.00.001,Subprefeitura São Mateus,777,Despesa Regionalizável,2,Leste,30,SAO MATEUS,0,Supra-Distrital,1,2760,Percentual de terminais urbanos de ônibus com ...,%,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
7225,52_10_3011_3002,52,Subprefeitura Vila Mariana,10,Administração da Subprefeitura,3011,"Modernização Tecnológica, Desburocratização e ...",3002,"Ampliação, Reforma e Requalificação de Prédios...",777.04.12.00.001,Subprefeitura Vila Mariana,777,Despesa Regionalizável,4,Sul,12,VILA MARIANA,0,Supra-Distrital,1,2608,Edificações reformadas/ampliadas,unida,"5,00",1000.0,"2,00

In [12]:
save_csv(df_ppa_reg, "ppa_reg.csv")

Base salva em data\ppa_reg.csv
